Run the cell below:

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import statsmodels.api as sm
import statsmodels.tsa.stattools as st
from linearmodels import PanelOLS

## Problem 1 
### (10 x 2 = 20 points)

The Fama-French five-factor model adds two new factors to the three-factor model: the RMW (robust minus weak) profitability factor and the CMA (conservative minus aggressive) investment factor. 

1. Download monthly returns from 1970 to 2020 (inclusive) for the Fama-French five factor model. This is the ``F-F_Research_Data_5_Factors_2x3`` file in the ``famafrench`` database (the first table in the dictionary returned by pandas_datareader gives you the monthly returns). Call this dataframe ``ff5f``. Divide all returns in this dataset by 100. Print the first 5 and last 5 rows in this dataframe.
2. Download monthly adjusted-close prices on AAPL from Yahoo Finance, from 2010 to 2020 (inclusive). Use these prices to calculate monthly returns. Call the returns dataframe ``aapl_ret``. Print the first 5 and last 5 rows in this dataframe.
3. Merge the ``ff5`` and ``aapl_ret`` dataframes using an inner join on date. Call this merged dataframe ``data``. Add a variable in this dataframe called ``const`` which equals 1 everywhere. Print the first 5 and last 5 rows of ``data``.
4. Regress monthly AAPL excess returns from 2010 to 2020 (inclusive) on the Fama-French five factors (Mkt-RF, SMB, HML, RMW, and CMA) and a constant. Print the regression table.

Use the print function to provide answers to the following questions (i.e. your answer should be the argument to the print function). All these answers should be based on the results obtained from the regression above.

5. Is AAPL mispriced with respect to the five-factor model at the 5% significance level? If so, is it overpriced or underpriced? How did you reach your conclusion?
6. Does AAPL have a significant exposure (at the 5% level) to either of the five factors? If so, which ones? How did you reach your conclusion?
7. Using the five-factor model, what percentage of AAPL total risk is systematic?
8. Using the five-factor model, what excess return do we expect AAPL to have if the returns on the market, SMB, HML, RMW, and CMA factors are 0.5% (i.e. 0.005), 0.2%, 0.3%, 0.6% and -0.1% respectively?
9. Use data from 1970 to 2020 to estimate the risk premia (expected excess returns) on the five factors. Print these out.
10. Using the estimated risk premia from point 9 above, calculate the risk premium on AAPL stock assuming the true alpha of AAPL is 0. 

In [2]:
# 1
print("Output for part 1:")

ff5f = pdr.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench', '1970-01-01', '2020-12-31')[0]/100
ff5f

Output for part 1:


,Mkt-RF,SMB,HML,RMW,CMA,RF
Date,,,,,,
1970-01,-0.0810,0.0313,0.0312,-0.0171,0.0385,0.0060
1970-02,0.0513,-0.0274,0.0393,-0.0232,0.0274,0.0062
1970-03,-0.0106,-0.0240,0.0399,-0.0101,0.0432,0.0057
1970-04,-0.1100,-0.0637,0.0617,-0.0069,0.0626,0.0050
1970-05,-0.0692,-0.0445,0.0332,-0.0125,0.0392,0.0053
...,...,...,...,...,...,...
2020-08,0.0763,-0.0087,-0.0293,0.0427,-0.0130,0.0001
2020-09,-0.0363,-0.0007,-0.0266,-0.0129,-0.0177,0.0001
2020-10,-0.0210,0.0467,0.0419,-0.0093,-0.0073,0.0001


In [3]:
# 2
print("Output for part 2:")

aapl_raw = yf.download('AAPL','2010-01-01','2020-12-31',
                       interval = '1mo')['Adj Close'].dropna()

aapl_ret = aapl_raw.to_frame().pct_change().dropna()
aapl_ret.rename(columns = {'Adj Close': 'AAPL'}, inplace = True)
aapl_ret.index = aapl_ret.index.to_period('M')
aapl_ret

Output for part 2:
[*********************100%***********************]  1 of 1 completed


,AAPL
Date,
2010-02,0.065396
2010-03,0.148470
2010-04,0.111022
2010-05,-0.016125
2010-06,-0.020827
...,...
2020-08,0.214379
2020-09,-0.100908
2020-10,-0.060012


In [4]:
# 3
print("Output for part 3:")

data = aapl_ret.join(ff5f)
data['const'] = 1
data

Output for part 3:


,AAPL,Mkt-RF,SMB,HML,RMW,CMA,RF,const
Date,,,,,,,,
2010-02,0.065396,0.0340,0.0151,0.0322,-0.0028,0.0140,0.0000,1
2010-03,0.148470,0.0631,0.0185,0.0221,-0.0063,0.0167,0.0001,1
2010-04,0.111022,0.0200,0.0498,0.0289,0.0070,0.0174,0.0001,1
2010-05,-0.016125,-0.0789,0.0004,-0.0244,0.0127,-0.0023,0.0001,1
2010-06,-0.020827,-0.0557,-0.0247,-0.0470,-0.0018,-0.0155,0.0001,1
...,...,...,...,...,...,...,...,...
2020-08,0.214379,0.0763,-0.0087,-0.0293,0.0427,-0.0130,0.0001,1
2020-09,-0.100908,-0.0363,-0.0007,-0.0266,-0.0129,-0.0177,0.0001,1
2020-10,-0.060012,-0.0210,0.0467,0.0419,-0.0093,-0.0073,0.0001,1


In [5]:
# 4
print("Output for part 4:")

res = sm.OLS(data['AAPL'] - data['RF'],
             data[['const','Mkt-RF','SMB','HML','RMW','CMA']],
            missing = 'drop').fit()
print(res.summary())

Output for part 4:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.456
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     20.96
Date:                Mon, 04 Apr 2022   Prob (F-statistic):           3.62e-15
Time:                        12:40:22   Log-Likelihood:                 188.04
No. Observations:                 131   AIC:                            -364.1
Df Residuals:                     125   BIC:                            -346.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0087      0.006 

In [6]:
# 5
print("Output for part 5:")
print("Question 5 answer: ...")
#print(AAPL is not misspriced because the alpha has a pvalue > 0.05')

Output for part 5:
Question 5 answer: ...


In [7]:
# 6
print("Output for part 6:")
print("Question 6 answer: ...")
#print('The factors on which AAPL loads significantly are the ones with a pvalue < 0.05 are: Mkt-RF and RMW')

Output for part 6:
Question 6 answer: ...


In [8]:
# 7
print("Output for part 7:")
print("Question 7 answer: ...")
#print('Percentage of AAPL risk that is systematic: ', res.rsquared)

Output for part 7:
Question 7 answer: ...


In [9]:
# 8
print("Output for part 8:")

cond_pred = res.params['const'] + 0.005 * res.params['Mkt-RF'] + 0.002 * res.params['SMB'] + \
            0.003 * res.params['HML'] + 0.006 * res.params['RMW'] -0.001 * res.params['CMA']
print('Conditional prediction = ', cond_pred)

Output for part 8:
Conditional prediction =  0.02198378176014695


In [10]:
# 9
print("Output for part 9:")

rp = ff5f.mean()
rp

Output for part 9:


Mkt-RF    0.005926
SMB       0.001631
HML       0.002530
RMW       0.002662
CMA       0.002972
RF        0.003723
dtype: float64

In [11]:
# 10
print("Output for part 10:")
uncond_pred = rp['Mkt-RF'] * res.params['Mkt-RF'] + rp['SMB'] * res.params['SMB'] + \
            rp['HML'] * res.params['HML'] + rp['RMW'] * res.params['RMW'] + rp['CMA'] * res.params['CMA']
print('Unconditional prediction = ', uncond_pred)

Output for part 10:
Unconditional prediction =  0.007097167517847282


## Problem 2 
### (5 x 8 = 40 points)

For this problem, you will be testing if returns are predictable at the annual level.

Data:
1. Load the CRSP dataset from the "crspm.zip" file and keep only the "permno", "date", and "ret" variables. Drop all rows with any missing values. Call the resulting dataframe "crsp" and print its first two rows.
    1. In "crsp", create a new variable "year" that extracts the year from the "date" variable (as an integer, not a period date). Print a table that shows only the min and max of the "year" variable.
    2. Calculate annual compounded returns for each firm, each year, and store those in a separate dataframe called "crsp_an". Print the first 5 rows of "crsp_an".
    
2. Load the Compustat dataset from the "compa.zip" file and keep only the "permno", "datadate", "sich" (industry identifier) and "at" (total assets) variables. Drop all rows with any missing values in this new dataframe. Keep only rows with strictly positive total assets ("at"). Call the resulting dataframe "comp".
    1. In "comp", create a new variable "year" that extracts the year from the "datadate" variable (as an integer, not a period date). Print a table that shows only the min and max of the "year" variable.
    2. In "comp", create a new variable "ag" as the annual percentage growth in total assets. Winsorize it at the 1 and 99 percentiles and call this variable "ag_w". Print a table that shows the mean and standard deviation of "ag" and "ag_w".
    3. In "comp", create a new variable "sic3" that contains the first three digits of the "sich" variable
    
3. Merge the resulting "comp" dataframe with the "crsp_an" dataframe based on "permno" and "year". Call the merged dataset "data" and print out its shape.
    1. Create a new variable called "future_ret" that, for every firm, every year, stores the annual returns of that firm in the following year. Print the first 5 rows of this dataframe.
    2. Set the index of "data" to be ("permno", "year"). Keep only then "future_ret", "ag_w", and "sic3" variables. Create a new variable called "const" that equals 1 everywhere. Print out the first 2 rows of this dataset.

Analysis:

4. Regress "future_ret", on current winsorized asset growth ("ag_w") and a constant. This should be simple OLS, with no adjustments for fixed effects or corrections to standard errors. Print your results (the regression table)
    1. Use the print function to answer the following question: "Based on these regression results, does current asset growth have statistically significant explanatory power over future returns? How did you reach that conclusion?"
    
5. Run the same regression as above, only this time add time fixed effects and industry ("sic3") fixed effects, and cluster your standard errors at the firm and year level (ignore the warnings produced in this step, they are harmless). Print your results (the regression table)
    1. Use the print function to answer the following question: "Based on these regression results, does current asset growth have statistically significant explanatory power over future returns? How did you reach that conclusion?"

In [12]:
# 1
print("Output for part 1:")

crsp = pd.read_pickle('../../lectures/data/crspm.zip')
crsp = crsp[['permno','date','ret']].dropna().copy()
crsp.head(2)

Output for part 1:


,permno,date,ret
1,10000.0,1986-02-28,-0.257143
2,10000.0,1986-03-31,0.365385


In [13]:
# 1.A
print("Output for part 1.A:")

crsp['year'] = pd.to_datetime(crsp['date']).dt.year
crsp['year'].agg(['min','max'])

Output for part 1.A:


min    1980
max    2020
Name: year, dtype: int64

In [14]:
# 1.B
print("Output for part 1.B:")

crsp['anret'] = 1 + crsp['ret']
crsp_an = crsp.groupby(['permno','year'])['anret'].prod() 
crsp_an = crsp_an.to_frame()
crsp_an.head()

Output for part 1.B:


anret
permno  year          
10000.0 1986  0.117857
        1987  0.424242
10001.0 1986  1.217368
        1987  0.898725
        1988  1.163160

In [15]:
# 2
print("Output for part 2:")

comp = pd.read_pickle('../../lectures/data/compa.zip')
comp = comp[['permno','datadate','sich','at',]].dropna().copy()
comp = comp.loc[comp['at']>0, :].copy()
comp.head(2)

Output for part 2:


,permno,datadate,sich,at
15,10031.0,1988-01-31,5712.0,16.042
16,10031.0,1989-01-31,5712.0,16.280


In [16]:
# 2.A
print("Output for part 2.A:")

comp['year'] = pd.to_datetime(comp['datadate']).dt.year
comp['year'].agg(['min','max'])

Output for part 2.A:


min    1982
max    2020
Name: year, dtype: int64

In [17]:
# 2.B
print("Output for part 2.B:")

comp.sort_values(['permno','year'], inplace = True)
comp['at_lag1'] = comp.groupby('permno')['at'].shift(1)

comp['ag'] = comp['at']/comp['at_lag1'] - 1
comp['ag_w'] = comp['ag'].clip(lower = comp['ag'].quantile(0.01), upper = comp['ag'].quantile(0.99))

comp[['ag','ag_w']].agg(['mean','std'])

Output for part 2.B:


,ag,ag_w
mean,2.706688,0.143621
std,1037.143277,0.451803


In [18]:
# 2.C
print("Output for part 2.C:")

comp['sic3'] = comp['sich'].astype("string").str[0:3]
comp.head()

Output for part 2.C:


,permno,datadate,sich,at,year,at_lag1,ag,ag_w,sic3
188566,10001.0,1987-06-30,4924.0,11.771,1987,NaN,NaN,NaN,492
188567,10001.0,1988-06-30,4924.0,11.735,1988,11.771,-0.003058,-0.003058,492
188568,10001.0,1989-06-30,4924.0,18.565,1989,11.735,0.582020,0.582020,492
188569,10001.0,1990-06-30,4924.0,18.881,1990,18.565,0.017021,0.017021,492
188570,10001.0,1991-06-30,4924.0,19.599,1991,18.881,0.038028,0.038028,492


In [19]:
# 3
print("Output for part 3:")

data = comp.merge(crsp_an, how = 'inner', on = ['permno','year'])
data.shape

Output for part 3:


(156258, 10)

In [20]:
# 3.A
print("Output for part 3.A:")

data.sort_values(['permno','year'], inplace=True)
data['future_ret'] = data.groupby('permno')['anret'].shift(-1)
data.head()

Output for part 3.A:


,permno,datadate,sich,at,year,at_lag1,ag,ag_w,sic3,anret,future_ret
0,10001.0,1987-06-30,4924.0,11.771,1987,NaN,NaN,NaN,492,0.898725,1.163160
1,10001.0,1988-06-30,4924.0,11.735,1988,11.771,-0.003058,-0.003058,492,1.163160,1.687923
2,10001.0,1989-06-30,4924.0,18.565,1989,11.735,0.582020,0.582020,492,1.687923,0.991279
3,10001.0,1990-06-30,4924.0,18.881,1990,18.565,0.017021,0.017021,492,0.991279,1.607471
4,10001.0,1991-06-30,4924.0,19.599,1991,18.881,0.038028,0.038028,492,1.607471,1.012620


In [21]:
# 3.B
print("Output for part 3.B:")

data.set_index(['permno','year'],inplace = True)
data = data[['future_ret','ag_w','sic3']].copy()
data['const'] = 1
data.head(2)

Output for part 3.B:


future_ret      ag_w sic3  const
permno  year                                  
10001.0 1987    1.163160       NaN  492      1
        1988    1.687923 -0.003058  492      1

In [22]:
# 4
print("Output for part 4:")

res_simple = sm.OLS(data['future_ret'],
                    data[['const','ag_w']],
                   missing = 'drop').fit()
print(res_simple.summary())

Output for part 4:
                            OLS Regression Results                            
Dep. Variable:             future_ret   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     515.0
Date:                Mon, 04 Apr 2022   Prob (F-statistic):          8.59e-114
Time:                        12:40:25   Log-Likelihood:            -1.7249e+05
No. Observations:              125401   AIC:                         3.450e+05
Df Residuals:                  125399   BIC:                         3.450e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1903      0.003 

In [23]:
# 4.A
print("Output for part 4.A:")
print("Question 4.A answer: ...")

#print("Yes, because the p-value of the AG variable is lower than 0.05:",res_simple.pvalues['ag_w'])

Output for part 4.A:
Question 4.A answer: ...


In [24]:
# 5
print("Output for part 5:")

mod_robust = PanelOLS(data['future_ret'], 
                      data[['const','ag_w']], 
                      time_effects = True, other_effects = data['sic3'])
res_robust = mod_robust.fit(cov_type = 'clustered', cluster_entity = True, cluster_time = True)
print(res_robust.summary)

Output for part 5:


/home/imb/anaconda3/lib/python3.9/site-packages/linearmodels/shared/exceptions.py:37: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:             future_ret   R-squared:                        0.0036
Estimator:                   PanelOLS   R-squared (Between):             -0.0149
No. Observations:              125401   R-squared (Within):               0.0072
Date:                Mon, Apr 04 2022   R-squared (Overall):              0.0041
Time:                        12:40:25   Log-likelihood                -1.678e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      445.92
Entities:                       13205   P-value                           0.0000
Avg Obs:                       9.4965   Distribution:                F(1,125091)
Min Obs:                       1.0000                                           
Max Obs:                       66.000   F-statistic (robust):             33.237
                            

In [25]:
# 5.A
print("Output for part 5.A:")
print("Question 5.A answer: ...")

#print("Yes, because the p-value of the AG variable is lower than 0.05:",res_robust.pvalues[1])

Output for part 5.A:
Question 5.A answer: ...


## Problem 3 
### (10 x 4 = 40 points)

For this problem we will analyze the effect of tresury yields on future unemployment rates.

1. Download data on monthly yields on ten-year treasury bonds (``GS10``) and the unemployment rate (``UNRATE``) from the St Louis Fred from 1960 to 2020 (inclusive). Rename ``GS10`` to ``10yrate`` and ``UNRATE`` to ``unemployment``. Plot these two variables on the same plot.
2. Create a variable called ``future_unemployment`` which gives us the unemployment rate from 12 months into the future. Create a variable called ``const`` which equals 1 everywhere. Print the first 5 and last 5 rows of this new version of your dataset.
3. Print the correlation between ``future_unemployment`` and ``10yrate``, as well as the 12-month autocorrelations of ```future_unemployment`` and ``10yrate``.
4. Regress ``future_unemployment`` on ``10yrate`` and a constant (simple regression with no corrections). Print the regression table.
5. Run the same regression as in point 4 above, but this time use an HAC covariance estimator with maximum 5 lags. Print the regression table.
6. Use an augmented Dickey-Fuller test to test if you can reject the null hypotheses that ``future_employment`` and ``10yrate`` are non-stationary at the 99\% confidence level.
7. Create a new variable called ``unemployment_change`` which equals ``future_unemployment`` minus its value from 12 months prior (i.e. the 12-month difference in ``future_unemployment``). Create a new variable called ``10yrate_change`` which equals ``10yrate`` minus its value from 12 months prior. Plot these variables on the same graph.
8. Use an augmented Dickey-Fuller test to test if you can reject the null hypotheses that ``unemployment_change`` and ``10yrate_change`` are non-stationary at the 99\% confidence level.
9. Regress ``unemployment_change`` on ``10y_rate`` and a constant. Use an HAC covariance estimator with maximum 5 lags. Print the regression table.
10. Based on the tests you performed above, do you conclude that treasury yields are a statistically significant predictor of future unemployment at the 99\% confidence level?

In [ ]:
# 1
print("Output for part 1:")

df = pdr.DataReader(['UNRATE','GS10'],'fred', '1960-01-01','2020-12-31')
df.rename(columns = {'UNRATE': 'unemployment', 'GS10': '10yrate'}, inplace = True)
df.plot();

Output for part 1:


In [ ]:
# 2
print("Output for part 2:")

df.sort_index(inplace = True)
df['future_unemployment'] = df['unemployment'].shift(-12)
df['const'] = 1
df

In [ ]:
# 3
print("Output for part 3:")

print("Correlation between 10-year rate and future unemployment = ",
      df.corr().loc['future_unemployment','10yrate'])

print("12-month autocorrelation in future unemployment = ",
      df['future_unemployment'].autocorr(12))

print("12-month autocorrelation in 10-year rate = ", df['10yrate'].autocorr(12))

In [ ]:
# 4
print("Output for part 4:")

res = sm.OLS(df['future_unemployment'],
            df[['const','10yrate']],
            missing = 'drop').fit()
print(res.summary())

In [ ]:
# 5
print("Output for part 5:")

res_robust = res.get_robustcov_results(cov_type = "HAC", maxlags = 5)
print(res_robust.summary())

In [ ]:
# 6
print("Output for part 6:")

future_unemployment_adf = st.adfuller(df['future_unemployment'].dropna())
print("P-value on ADF test for future unemloyment = ", future_unemployment_adf[1])

rate_adf = st.adfuller(df['10yrate'].dropna())
print("P-value on ADF test for 10-year rate = ", rate_adf[1])

In [ ]:
print("Part 6 answer: ... ")
#print("Both of the p-values above are higher than 1%, so we can not reject non-stationarity for either variable.")

In [ ]:
# 7
print("Output for part 7:")

df.sort_index(inplace=True)
df['unemployment_change'] = df['future_unemployment'].diff(12)
df['10yrate_change'] = df['10yrate'].diff(12)
df[['unemployment_change','10yrate_change']].plot();

In [ ]:
# 8
print("Output for part 8:")

change_unemployment_adf = st.adfuller(df['unemployment_change'].dropna())
print("P-value on ADF test for unemployment change = ", change_unemployment_adf[1])

change_rate_adf = st.adfuller(df['10yrate_change'].dropna())
print("P-value on ADF test for 10-year rate change = ", change_rate_adf[1])

In [ ]:
print("Part 8 answer: ... ")
#print("Both of the p-values above are lower than 1%, so we can reject non-stationarity for both variables.")

In [ ]:
# 9
print("Output for part 9:")

res_adf = sm.OLS(df['unemployment_change'],
            df[['const','10yrate_change']],
            missing = 'drop').fit()
res_robust_adf = res_adf.get_robustcov_results(cov_type = "HAC", maxlags = 5)
print(res_robust_adf.summary())

In [ ]:
# 10
print("Output for part 10:")
print("Part 10 answer: ... ")

#print("""Because the levels regression suffers from non-stationarity, and the differenced regression does not,
#      we need to draw our conclusion based on the last regression. There, the p-value for the treasury rate is not 
#      lower than 1%, so we can not conclude that the rate has significant explanatory power over future unemployment.""") 